# Fine-Tuning NER Model


### Import the necessary libraries

In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

In [ ]:
!pip install seqeval

In [8]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import evaluate


# # Configure logging
# logging.basicConfig(filename='../logs/data_processing.log',
#                     level=logging.INFO,
#                     format='%(asctime)s:%(levelname)s:%(message)s')


# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
# # Import the module
# from ner_finetuning import *
# from load_data import Load_Data

In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
!cp /content/drive/MyDrive/merged_amharic_ner_data.conll /content/


In [11]:
def load_conll_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = f.read().strip().split('\n\n')

    sentences, labels = [], []
    for sentence in data:
        words, tags = [], []
        for line in sentence.split('\n'):
            word, tag = line.split()
            words.append(word)
            tags.append(tag)
        sentences.append(words)
        labels.append(tags)

    return sentences, labels

### Load the labeled data

In [12]:
# Create an instance of CSVReader
file_path = ('/content/drive/MyDrive/merged_amharic_ner_data.conll')
sentences, labels = load_conll_data(file_path)
print(f"Loaded {len(sentences)} sentences.")


Loaded 6454 sentences.


In [13]:
#Step 4: Prepare the Dataset
def prepare_dataset(sentences, labels):
    df = pd.DataFrame({'tokens': sentences, 'ner_tags': labels})
    dataset = Dataset.from_pandas(df)
    return dataset

dataset = prepare_dataset(sentences, labels)


In [14]:
# Step 5: Define label mappings (label to id and id to label)
def get_label_encodings(labels):
    unique_labels = set()
    for label_list in labels:
        unique_labels.update(label_list)
    label_list = sorted(unique_labels)
    label2id = {label: idx for idx, label in enumerate(label_list)}
    id2label = {idx: label for label, idx in label2id.items()}
    return label_list, label2id, id2label

label_list, label2id, id2label = get_label_encodings(labels)
num_labels = len(label_list)
print("Labels:", label_list)


Labels: ['B-LOC', 'B-PRICE', 'B-PROD', 'B-PRODUCT', 'B-Price', 'B-Product', 'I-LOC', 'I-PRICE', 'I-PRODUCT', 'I-Price', 'I-Product', 'O']


In [15]:
# Step 6: Load the pre-trained model and tokenizer
model_name = "xlm-roberta-base"  # Change to your preferred model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/6454 [00:00<?, ? examples/s]

In [17]:
# Step 8: Set up the trainer
# Convert to Hugging Face DatasetDict
from datasets import DatasetDict

dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset['train']
eval_dataset = dataset['test']

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(eval_dataset)}")


Training samples: 5163
Validation samples: 1291


In [18]:
# Step 9: Set Up Training Arguments

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",              # Output directory
    eval_strategy="epoch",               # Updated to use eval_strategy instead of deprecated evaluation_strategy
    save_strategy="epoch",               # Ensuring save strategy matches eval strategy
    learning_rate=2e-5,                  # Learning rate
    per_device_train_batch_size=16,      # Batch size for training
    per_device_eval_batch_size=16,       # Batch size for evaluation
    num_train_epochs=3,                  # Number of epochs
    weight_decay=0.01,                   # Weight decay
    load_best_model_at_end=True,         # Load the best model at the end of training
)



In [19]:
# Step 10: Define Evaluation Metrics
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_labels = [[id2label[label_id] for label_id in label if label_id != -100] for label in labels]
    true_predictions = [[id2label[pred_id] for pred_id, label_id in zip(prediction, label) if label_id != -100]
                        for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [20]:
#Step 11: Initialize the Trainer
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [21]:
#Step 12: Train the Model
trainer.train()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.064980,0.862741,0.860446,0.861592,0.981759
2,0.190600,0.035679,0.919936,0.935748,0.927774,0.990448
3,0.190600,0.027972,0.940661,0.943933,0.942294,0.992898


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=969, training_loss=0.11448625538986411, metrics={'train_runtime': 986.8133, 'train_samples_per_second': 15.696, 'train_steps_per_second': 0.982, 'total_flos': 2151761009916672.0, 'train_loss': 0.11448625538986411, 'epoch': 3.0})

In [22]:
# Step 13: Evaluate the Model
evaluation_results = trainer.evaluate()
print(evaluation_results)


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.027972057461738586, 'eval_precision': 0.9406606851549756, 'eval_recall': 0.9439328831594025, 'eval_f1': 0.9422939434174242, 'eval_accuracy': 0.9928977491565114, 'eval_runtime': 24.7191, 'eval_samples_per_second': 52.227, 'eval_steps_per_second': 3.277, 'epoch': 3.0}


In [23]:
#Step 14: Save the Model
model.save_pretrained("./fine_tuned_ner_model")
tokenizer.save_pretrained("./fine_tuned_ner_model")


('./fine_tuned_ner_model/tokenizer_config.json',
 './fine_tuned_ner_model/special_tokens_map.json',
 './fine_tuned_ner_model/sentencepiece.bpe.model',
 './fine_tuned_ner_model/added_tokens.json',
 './fine_tuned_ner_model/tokenizer.json')

In [24]:
def predict_ner(text, model, tokenizer, id2label):
    # Set the model to evaluation mode
    model.eval()

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Move inputs to the same device as the model
    device = next(model.parameters()).device
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)

    # Get the predicted labels
    predictions = torch.argmax(outputs.logits, dim=2)

    # Convert predictions to labels
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    predicted_labels = [id2label[prediction.item()] for prediction in predictions[0]]

    # Return tokens and their corresponding predicted labels
    return list(zip(tokens, predicted_labels))


In [25]:
# Sample input text
sample_text = "ምርቶች ከአዲስ አበባ በቅናሽ ዋጋ ይሰጣሉ።"

# Call the predict function
predictions = predict_ner(sample_text, model, tokenizer, id2label)

# Print predictions
for token, label in predictions:
    print(f"{token}: {label}")


<s>: O
▁ምርቶች: O
▁ከአዲስ: O
▁አበባ: O
▁በ: O
ቅና: O
ሽ: O
▁ዋጋ: O
▁ይሰ: O
ጣ: O
ሉ።: O
</s>: O
